In [1]:
!pip install openpyxl transformers

In [2]:
import pandas as pd
from IPython.display import display

def read_data_frame(file_path, sheet_name):
    """Read a specific sheet from an Excel file."""
    return pd.read_excel(file_path, sheet_name=sheet_name)

# Path to the Excel file
file_path = '/content/Solving Business Problems with AI.xlsx'

# Reading specific sheets from the Excel file
products_df = read_data_frame(file_path, 'products')
emails_df = read_data_frame(file_path, 'emails')

# Display the first 3 rows of each DataFrame
display(products_df.head(3))
display(emails_df.head(3))


,product_id,name,category,description,stock,seasons,price
0,RSG8901,Retro Sunglasses,Accessories,Transport yourself back in time with our retro...,1,"Spring, Summer",26.99
1,SWL2345,Sleek Wallet,Accessories,Keep your essentials organized and secure with...,5,All seasons,30.00
2,VSC6789,Versatile Scarf,Accessories,Add a touch of versatility to your wardrobe wi...,6,"Spring, Fall",23.00


,email_id,subject,message
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT..."
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V..."
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and..."


In [3]:
from transformers import pipeline

# Load a pre-trained Hugging Face model for text classification
classifier = pipeline('text-classification', model='distilbert-base-uncased-finetuned-sst-2-english',device_map={"":0})

def classify_email(subject, message):
    # Create the input text
    email_text = f"Subject: {subject}\nMessage: {message}"

    # Call the Hugging Face classifier
    result = classifier(email_text)

    # Check the label and return the appropriate category
    label = result[0]['label'].lower()
    if 'positive' in label:  # Assuming the model outputs positive/negative labels
        return 'product_inquiry'
    elif 'negative' in label:
        return 'order_request'
    else:
        return 'uncertain'

# Apply the classification to your DataFrame
emails_df['category'] = emails_df.apply(lambda row: classify_email(row['subject'], row['message']), axis=1)

# Display the refined classification results
display(emails_df[['email_id', 'category']])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency ple

,email_id,category
0,E001,product_inquiry
1,E002,product_inquiry
2,E003,order_request
3,E004,product_inquiry
4,E005,order_request
5,E006,order_request
6,E007,order_request
7,E008,product_inquiry
8,E009,order_request
9,E010,product_inquiry


In [4]:
import pandas as pd
import re

def extract_order_details(email_text):
    """
    Extract product IDs and quantities from the email content.
    Assumes format: "Product ID: XXXX, Quantity: XX"
    """
    pattern = r"Product ID: (\w+), Quantity: (\d+)"
    matches = re.findall(pattern, email_text)
    return [{'product_id': match[0], 'quantity': int(match[1])} for match in matches]

def process_order(email_id, email_text, products_df):
    """
    Process each order by checking stock, updating it, and generating order status and responses.
    """
    order_details = extract_order_details(email_text)
    order_status = []
    responses = []

    for detail in order_details:
        product_id = detail['product_id']
        requested_quantity = detail['quantity']
        product_info = products_df.loc[products_df['product_id'] == product_id]

        if not product_info.empty:
            stock = product_info['stock'].values[0]

            if stock >= requested_quantity:
                # Update the stock in the DataFrame
                new_stock = stock - requested_quantity
                products_df.loc[products_df['product_id'] == product_id, 'stock'] = new_stock
                status = 'created'
                response = f"Your order for {product_info['name'].values[0]} has been processed successfully."
            else:
                status = 'out of stock'
                response = f"Sorry, {product_info['name'].values[0]} is out of stock for the requested quantity of {requested_quantity}."
        else:
            status = 'product not found'
            response = f"Product ID: {product_id} is not found in our catalog."

        order_status.append({'email_id': email_id, 'product_id': product_id, 'quantity': requested_quantity, 'status': status})
        responses.append({'email_id': email_id, 'response': response})

    return order_status, responses

# Sample email content for testing
email_text_example = "Product ID: RSG8901, Quantity: 1"
order_status, order_responses = process_order('E001', email_text_example, products_df)

# Converting the results to DataFrame for better visualization and manipulation
order_status_df = pd.DataFrame(order_status)
order_responses_df = pd.DataFrame(order_responses)

# Displaying the results
order_status_df, order_responses_df

(  email_id product_id  quantity   status
 0     E001    RSG8901         1  created,
   email_id                                           response
 0     E001  Your order for Retro Sunglasses has been proce...)

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


def extract_keywords(text):
    return text.split()

# Function to find relevant products based on keywords
def find_relevant_products(keywords):
    # Convert product descriptions to a matrix of TF-IDF features
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(products_df['description'])

    # Construct a query from keywords
    query_vect = tfidf_vectorizer.transform([' '.join(keywords)])

    # Compute cosine similarity between the query and all product descriptions
    cos_similarities = cosine_similarity(query_vect, tfidf_matrix).flatten()

    # Get the top 5 most similar products
    top_product_indices = cos_similarities.argsort()[-5:][::-1]
    top_products = products_df.iloc[top_product_indices]

    return top_products

# Function to generate a response based on found products
def generate_response(email_id, inquiry):
    keywords = extract_keywords(inquiry)
    relevant_products = find_relevant_products(keywords)
    responses = []
    for index, product in relevant_products.iterrows():
        response = f"The {product['name']} is available for ${product['price']}. It's perfect for {product['seasons']} and here's more about it: {product['description']}"
        responses.append(response)

    # Create DataFrame for output
    response_df = pd.DataFrame({
        'email_id': [email_id] * len(responses),
        'response': responses
    })
    return response_df

In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


def extract_keywords(text):
    return text.split()

# Function to find relevant products based on keywords
def find_relevant_products(keywords):
    # Convert product descriptions to a matrix of TF-IDF features
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(products_df['description'])

    # Construct a query from keywords
    query_vect = tfidf_vectorizer.transform([' '.join(keywords)])

    # Compute cosine similarity between the query and all product descriptions
    cos_similarities = cosine_similarity(query_vect, tfidf_matrix).flatten()

    # Get the top 5 most similar products
    top_product_indices = cos_similarities.argsort()[-5:][::-1]
    top_products = products_df.iloc[top_product_indices]

    return top_products

# Function to generate a response based on found products
def generate_response(email_id, inquiry):
    keywords = extract_keywords(inquiry)
    relevant_products = find_relevant_products(keywords)
    responses = []
    for index, product in relevant_products.iterrows():
        response = f"The {product['name']} is available for ${product['price']}. It's perfect for {product['seasons']} and here's more about it: {product['description']}"
        responses.append(response)

    # Create DataFrame for output
    response_df = pd.DataFrame({
        'email_id': [email_id] * len(responses),
        'response': responses
    })
    return response_df

In [8]:
# Example usage
email_id = 'email234'
inquiry = "I'm looking for something nice for winter mornings"
response_df = generate_response(email_id, inquiry)
response_df.to_csv('inquiry_responses.csv', index=False)

print("Tabular format of the CSV file:")
print(response_df)

Tabular format of the CSV file:
   email_id                                           response
0  email234  The Long Fleece Robe is available for $38.0. I...
1  email234  The Striped Tee is available for $24.0. It's p...
2  email234  The Cozy Pajama Set is available for $27.0. It...
3  email234  The Infinity Scarf is available for $28.0. It'...
4  email234  The Puffer Parka is available for $38.0. It's ...
